In [1]:
#!pip install "gymnasium[mujoco]"
import os
import gymnasium as gym
import numpy as np , pandas as pd

# Create simulation environment
env = gym.make("Walker2d-v5", render_mode="ansi")

#Action Space : Box(-1.0, 1.0, (6,), float32)
#Observation Space : Box(-inf, inf, (17,), float64)
print(f"Action space: {env.action_space}")
print(f"Sample action: {env.action_space.sample()}")
print(f"Observation space: {env.observation_space}")
print(f"Sample observation: {env.observation_space.sample()}")

Action space: Box(-1.0, 1.0, (6,), float32)
Sample action: [ 0.22949143  0.04289797 -0.67640305 -0.03365321 -0.34958354  0.99556535]
Observation space: Box(-inf, inf, (17,), float64)
Sample observation: [ 1.03211765e-01 -2.06571544e-03 -1.67016068e+00 -4.15456372e-01
  5.45120322e-01 -6.46821001e-01  5.62315921e-01  5.60605487e-01
 -1.46569749e+00 -1.07029072e+00  1.29254757e-02  2.11310877e+00
 -6.02659190e-01  4.98582729e-01  1.99430696e-01  3.87812520e-02
  9.60141817e-01]


C:\MyVirtualEnvs\OpenAIGym\Lib\site-packages\gymnasium\envs\registration.py:728: UserWarning: WARN: The environment is being initialised with render_mode='ansi' that is not in the possible render_modes (['human', 'rgb_array', 'depth_array', 'rgbd_tuple']).
  logger.warn(


In [2]:
#Store inputs and outputs
action_headers = [
    'thigh_joint',           # Torque applied on the right thigh rotor [-1, 1]
    'leg_joint',             # Torque applied on the right leg rotor [-1, 1]
    'foot_joint',            # Torque applied on the right foot rotor [-1, 1]
    'thigh_left_joint',      # Torque applied on the left thigh rotor [-1, 1]
    'leg_left_joint',        # Torque applied on the left leg rotor [-1, 1]
    'foot_left_joint'        # Torque applied on the left foot rotor [-1, 1]
]
observation_headers = [
    'rootz',                 # z-coordinate of the torso (height)
    'rooty',                 # angle of the torso
    'thigh_joint',           # angle of the right thigh joint
    'leg_joint',             # angle of the right leg joint
    'foot_joint',            # angle of the right foot joint
    'thigh_left_joint',      # angle of the left thigh joint
    'leg_left_joint',        # angle of the left leg joint
    'foot_left_joint',       # angle of the left foot joint
    'rootx_vel',             # velocity of the x-coordinate of the torso
    'rootz_vel',             # velocity of the z-coordinate (height) of the torso
    'rooty_angvel',          # angular velocity of the torso angle
    'thigh_joint_angvel',    # angular velocity of the right thigh hinge
    'leg_joint_angvel',      # angular velocity of the right leg hinge
    'foot_joint_angvel',     # angular velocity of the right foot hinge
    'thigh_left_joint_angvel',  # angular velocity of the left thigh hinge
    'leg_left_joint_angvel',    # angular velocity of the left leg hinge
    'foot_left_joint_angvel'    # angular velocity of the left foot hinge
]
in_obs_headers = [ "in_"+h for h in observation_headers]
out_obs_headers = [ "out_"+h for h in observation_headers]
col_headers = in_obs_headers+action_headers+out_obs_headers

data = pd.DataFrame(columns=col_headers)

In [3]:
num_episodes=0
while len(data) < 1000:
    num_episodes += 1
    in_obs, info = env.reset()
    terminated, truncated = False , False
    while not (terminated or truncated):
        action = env.action_space.sample()  # Random action
        out_obs, reward, terminated, truncated, info = env.step(action) #output of action
        new_row =list(in_obs) + list(action) + list(out_obs)
        data.loc[len(data)] = new_row       #append
        #print(list(data.iloc[-1,:]))
        #previous step output is next step input
        in_obs = out_obs

print("Number of episodes simulated :",num_episodes)
print("Number of steps simulated :",len(data))
data.info()
data.describe()

Number of episodes simulated : 47
Number of steps simulated : 1007
<class 'pandas.DataFrame'>
Index: 1007 entries, 0 to 1006
Data columns (total 40 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   in_rootz                     1007 non-null   float64
 1   in_rooty                     1007 non-null   float64
 2   in_thigh_joint               1007 non-null   float64
 3   in_leg_joint                 1007 non-null   float64
 4   in_foot_joint                1007 non-null   float64
 5   in_thigh_left_joint          1007 non-null   float64
 6   in_leg_left_joint            1007 non-null   float64
 7   in_foot_left_joint           1007 non-null   float64
 8   in_rootx_vel                 1007 non-null   float64
 9   in_rootz_vel                 1007 non-null   float64
 10  in_rooty_angvel              1007 non-null   float64
 11  in_thigh_joint_angvel        1007 non-null   float64
 12  in_leg_joint_angvel      

,in_rootz,in_rooty,in_thigh_joint,in_leg_joint,in_foot_joint,in_thigh_left_joint,in_leg_left_joint,in_foot_left_joint,in_rootx_vel,in_rootz_vel,...,out_foot_left_joint,out_rootx_vel,out_rootz_vel,out_rooty_angvel,out_thigh_joint_angvel,out_leg_joint_angvel,out_foot_joint_angvel,out_thigh_left_joint_angvel,out_leg_left_joint_angvel,out_foot_left_joint_angvel
count,1007.000000,1007.000000,1007.000000,1007.000000,1007.000000,1007.000000,1007.000000,1007.000000,1007.000000,1007.000000,...,1007.000000,1007.000000,1007.000000,1007.000000,1007.000000,1007.000000,1007.000000,1007.000000,1007.000000,1007.000000
mean,1.190522,-0.368588,-0.212171,-0.370671,0.282594,-0.174971,-0.395054,0.213448,-0.938056,-1.091662,...,0.232139,-0.983431,-1.242179,-5.301503,-3.177889,-3.889803,1.458053,-2.848699,-4.240196,1.299146
std,0.074141,0.304464,0.264229,0.507420,0.427948,0.257372,0.476418,0.428113,0.541525,1.103988,...,0.439679,0.508780,1.196483,4.301169,5.757823,5.041754,7.632353,5.837198,5.094113,7.292040
min,0.812558,-0.998920,-1.123795,-2.294313,-1.115126,-1.149394,-1.978839,-1.004201,-2.327169,-6.179489,...,-1.004201,-2.327169,-6.287608,-10.000000,-10.000000,-10.000000,-10.000000,-10.000000,-10.000000,-10.000000
25%,1.159018,-0.606740,-0.370366,-0.561088,-0.007605,-0.275513,-0.645399,-0.054793,-1.326221,-1.669407,...,-0.063999,-1.335281,-1.833634,-10.000000,-9.582737,-8.881037,-5.572786,-9.156989,-9.426427,-4.614213
50%,1.212499,-0.327323,-0.123411,-0.130667,0.213318,-0.076322,-0.188089,0.098896,-0.972472,-0.823196,...,0.151116,-0.996467,-0.973173,-5.830452,-3.336961,-4.233957,2.441868,-2.649540,-5.125390,1.726567
75%,1.246404,-0.072139,-0.001611,-0.004543,0.672381,0.002419,-0.016265,0.515214,-0.566363,-0.259475,...,0.583177,-0.637129,-0.360496,-2.085887,0.568885,0.058594,10.000000,0.777101,-0.002112,9.915740
max,1.283598,0.024142,0.148073,0.109997,1.161146,0.150938,0.123584,1.086118,0.619903,0.663708,...,1.086118,0.619903,0.663708,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000


In [4]:
data.to_csv('data.csv', index=False)
env.close()

In [6]:
###
# DATA PREPROCESSING
###
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import time

df = pd.read_csv('data.csv')
x_data = df.iloc[:, :23].values
y_data = df.iloc[:, 23:40].values

x_tr, x_te, y_tr, y_te = train_test_split(
    x_data, y_data, test_size=0.2, random_state=316
)

In [10]:
mdl_dict = {
    'RandFor': MultiOutputRegressor(
        RandomForestRegressor(n_estimators=100, max_depth=10, random_state=316 , verbose=2)
    ),
    'GradBst': MultiOutputRegressor(
        GradientBoostingRegressor(n_estimators=100, max_depth=6, random_state=316 , verbose=2)
    ),
    'Ridge': Ridge(alpha=1.0, random_state=316)
}

res_list = []
#Training various models
for mdl_name, mdl in mdl_dict.items():
    t_start = time.time()
    mdl.fit(x_tr, y_tr)
    tr_time = time.time() - t_start
    
    y_pred = mdl.predict(x_te)
    
    rmse_val = np.sqrt(mean_squared_error(y_te, y_pred))
    r2_val = r2_score(y_te, y_pred)
    
    res_list.append({
        'Model': mdl_name,
        'RMSE': rmse_val,
        'R2': r2_val,
        'Time': tr_time
    })

res_df = pd.DataFrame(res_list)

building tree 1 of 100
building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
b

[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    0.6s


building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 of 100
building tree 55 of 100
building tree 56 of 100
building tree 57 of 100
building tree 58 of 100
building tree 59 of 100
building tree 60 of 100
building tree 61 of 100
building tree 62 of 100
building tree 63 of 100
building tree 64 of 100
building tree 65 of 100
building tree 66 of 100
building tree 67 of 100
building tree 68 of 100
building tree 69 of 100
building tree 70 of 100
building tree 71 of 100
building tree 72 of 100
building tree 73 of 100
building tree 74 of 100
building tree 75 of 100
building tree 76 of 100
building tree 77 of 100
building tree 78 of 100
building tree 79 of 100
building tree 80 of 100
building tree 81 of 100
building tree 82 of 100
building tree 83 of 100
building tree 84 of 100
building tree 85 of 100
building tree 86 of 100
building tree 87 of 100
building tree 88 of 100
building tree 89

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    1.7s finished


building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100


...

building tree 1 of 100
building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100


[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    0.5s


building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 of 100
building tree 55 of 100
building tree 56 of 100
building tree 57 of 100
building tree 58 of 100
building tree 59 of 100
building tree 60 of 100
building tree 61 of 100
building tree 62 of 100
building tree 63 of 100
building tree 64 of 100
building tree 65 of 100
building tree 66 of 100
building tree 67 of 100
building tree 68 of 100
building tree 69 of 100
building tree 70 of 100
building tree 71 of 100
building tree 72 of 100
building tree 73 of 100
building tree 74 of 100
building tree 75 of 100
building tree 76 of 100
building tree 77 of 100
building tree 78 of 100
building tree 79 of 100
building tree 80 of 100
building tree 81 of 100
building tree 82

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    1.7s finished
[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_j

      Iter       Train Loss   Remaining Time 
         1           0.0048            2.02s
         2           0.0039            1.66s
         3           0.0031            1.76s
         4           0.0025            1.65s
         5           0.0021            1.62s
         6           0.0017            1.67s
         7           0.0013            1.62s
         8           0.0011            1.59s
         9           0.0009            1.58s
        10           0.0007            1.59s
        11           0.0006            1.53s
        12           0.0005            1.56s
        13           0.0004            1.55s
        14           0.0003            1.50s
        15           0.0003            1.49s
        16           0.0002            1.47s
        17           0.0002            1.46s
        18           0.0001            1.42s
        19           0.0001            1.44s
        20           0.0001            1.42s
        21           0.0001            1.39s
        2

In [11]:
print(res_df)

     Model      RMSE        R2       Time
0  RandFor  1.367068  0.930770  29.577138
1  GradBst  1.320690  0.938226  32.129302
2    Ridge  1.699513  0.906552   0.040054


In [12]:
def topsis(mat, wts, impact):
    norm_mat = mat / np.sqrt((mat ** 2).sum(axis=0))
    wt_mat = norm_mat * wts
    
    id_best = np.zeros(wt_mat.shape[1])
    id_worst = np.zeros(wt_mat.shape[1])
    
    for i in range(wt_mat.shape[1]):
        if impact[i] == '+':
            id_best[i] = wt_mat[:, i].max()
            id_worst[i] = wt_mat[:, i].min()
        else:
            id_best[i] = wt_mat[:, i].min()
            id_worst[i] = wt_mat[:, i].max()
    
    s_best = np.sqrt(((wt_mat - id_best) ** 2).sum(axis=1))
    s_worst = np.sqrt(((wt_mat - id_worst) ** 2).sum(axis=1))
    
    score = s_worst / (s_best + s_worst)
    
    return score


met_cols = ['RMSE', 'R2', 'Time']
tps_data = res_df[met_cols].values

wts = np.array([0.4, 0.4 , 0.2])
impact = ['-', '+', '-']

tps_score = topsis(tps_data, wts, impact)

res_df['Score'] = tps_score
res_df['Rank'] = res_df['Score'].rank(ascending=False).astype(int)

res_df = res_df.sort_values('Rank').reset_index(drop=True)

print(res_df)
print(f"\nTop: {res_df.iloc[0]['Model']}")

res_df.to_csv('mdl_cmp.csv', index=False)


     Model      RMSE        R2       Time     Score  Rank
0    Ridge  1.699513  0.906552   0.040054  0.710257     1
1  GradBst  1.320690  0.938226  32.129302  0.289743     2
2  RandFor  1.367068  0.930770  29.577138  0.284183     3

Top: Ridge
